# Reinforcement Learning Algorithms
This is an exercise notebook for Computer Network Performance class of 14-12-2020.

In [1]:
import numpy as np

In [2]:
# a map of the labels for the actions and the states 
names = {"actions": {0: "maintain", 1:"ignore"},
         "states":{0: "good state", 1:"decay", 2:"broken"}}

# transition probability function S x S x A
T = [[[.9,0,.1],
      [.9,.1,0],
      [.2,0,.8]],

      [[.5,.5,0],
       [0,.5,.5],
       [0,0,1]]]

# rewards function S X A
R = [[1,2],
     [1,2],
     [-1,0]]

gamma = 0.9         # discount factor
threshold = 10**-5  # convergence threshold 

n_states = len(T[0])
n_actions = len(T)

## Value Iteration

In [3]:
def value_iteration(T, R, n_states, n_actions, gamma, threshold, verbose=False):
    """Value iteration algorithm. Given an MDP, it outputs an optimal policy."""
    
    V = [0] * n_states  # optimal values 
    n = 0               # iterations count 
    while True:
        n += 1
        delta_epoch = 0  # how much has changed since the last epoch (triggers halt after convergence)
        
        for s1 in range(n_states):
            temp = V[s1]  # old values
            
            # a value for s1, one for every action
            value_actions = [ R[s1][a] + sum([T[a][s1][s2] * gamma * V[s2] 
                                              for s2 in range(n_states)]) 
                                                  for a in range(len(T)) ]
            V[s1] = np.max(value_actions)
            
            delta_epoch = max(delta_epoch, abs(temp - V[s1]))
            
        if verbose: 
            print("iter number: ", n, "values: ", V)
            
        if delta_epoch < threshold:  # convergence check
            break
    
    policy = [-1]*n_states
    for s in range(n_states):
        opt_value_actions = [ R[s][a] + sum([T[a][s][s2] * gamma * V[s2] 
                                             for s2 in range(n_states)])
                                                 for a in range(len(T)) ]
        action = np.argmax(opt_value_actions)
        policy[s] = action

    return policy

def policy_to_string(plicy, names_map):
    """ Given a policy, a list of action indicies, it returns a list of action labels. """
    return [names["actions"][val] for val in policy]

In [4]:
policy = value_iteration(T, R, n_states, n_actions, gamma, threshold, verbose=False)
policy_readable = policy_to_string(policy, names)
print("Optimal policy for the given MDP is:\n" + str(policy_readable))

Optimal policy for the given MDP is:
['ignore', 'maintain', 'maintain']


## Policy Iteration

In [5]:
def policy_iteration(T, R, n_states, n_actions, gamma, threshold, verbose=False):
    """Policy iteration algorithm. Given an MDP, it outputs an optimal policy."""
    
    V = [0] * n_states       # optimal values 
    policy = [0] * n_states  # optimal policy
    n = 0

    while True:
        while True:
            n += 1
            delta_epoch = 0 # how much the values have changed since the last epoch (triggers halt after convergence)
            
            for s1 in range(n_states):
                temp = V[s1]
                a = policy[s1] # action of the policy under evaluation
                V[s1] = R[s1][a] + sum([T[a][s1][s2] * gamma * V[s2] for s2 in range(n_states)])
                
                delta_epoch = max(delta_epoch, abs(temp - V[s1]))

            if verbose: 
                print("iter number: ", n, "values: ", V)
                
            if delta_epoch < threshold:  # values convergence check
                break

        is_stable = True  # if the policy changed since the last epoch (triggers halt after convergence)
        for s in range(n_states):
            temp = policy[s]
            # a value for s, one for every action
            vals = [ R[s][a] + sum([T[a][s][s2] * gamma * V[s2] 
                                    for s2 in range(n_states)]) 
                                        for a in range(len(T)) ]
            action = np.argmax(vals)
            policy[s] = action

            if temp != policy[s]:
                is_stable = False

        if is_stable == True:  # policy convergence check
            break
            
    return policy

In [6]:
policy = policy_iteration(T, R, n_states, n_actions, gamma, threshold, verbose=False)
policy_readable = policy_to_string(policy, names)
print("Optimal policy for the given MDP is:\n" + str(policy_readable))

Optimal policy for the given MDP is:
['ignore', 'maintain', 'maintain']


## Q-learning

In [7]:
# a map of the labels for the actions and the states 
names = {"actions": {0:"go 0", 1:"go 1", 2:"go 2", 3:"go 3", 4:"go 4", 5:"go OUT"},
         "states":{0:"room 0", 1:"room 1", 2:"room 2", 3:"room 3", 4:"room 4", 5:"OUT"}}

# rewards function S X A
R = [[-1,-1,-1,-1,0,-1],
     [-1,-1,-1,0,-1,100],
     [-1,-1,-1,0,-1,-1],
     [-1,0,0,-1,0,-1],
     [0,-1,-1,0,-1,100],
     [-1,0,-1,-1,0,100]]

R = np.array(R)  # let is be a numpy array for convenience

gamma = 0.8         # discount factor
threshold = 10**-5  # convergence threshold 
alfa = 1            # learning rate

epsilon = 1        # probability (decaying) to get a random action
decay = 0.0005      

n_states = len(R)
n_actions = len(R[0])

end_state = 5

In [8]:
def qlearning(R, end_state, n_states, n_actions, gamma, alfa, threshold, epsilon, decay, seed=0, verbose=False):
    """Policy iteration algorithm. Given an MDP, it outputs an optimal policy."""
    
    np.random.seed(seed) # to set randomness
    Q = np.array([[0]* n_states] * n_states)    # optimal q values 

    while True:
        s1 = np.random.randint(n_states)  # choose a random state to start this epoch
        s2 = None                         # the target state is still unknown, need to choose and action first

        reached_end_epoch = False 
        delta_epoch = 0
        while not reached_end_epoch: # stop if the node was an ending node
            
            if s2 == end_state:
                reached_end_epoch = True
            
            available_actions_s1 = np.where(R[s1]>=0)[0] # indicies of non null actions from s1
            
            # epsilon probability gets smaller with time, encourage exploitation rather than exploration 
            if flip_bised_coin(epsilon): 
                # explore
                a = np.random.choice(available_actions_s1, 1)[0]  # actions randomly selected
            else:
                # exploit
                # its the action with the max Q from s1 if there are positive Q, otherwise a random available action  
                a = np.argmax(Q[s1]) if not  (Q[s1] == 0).all() else np.random.choice(available_actions_s1, 1)[0]
            
            epsilon = epsilon - decay # decay probability of random action
            
            s2 = a           # a is both the action and the future state
            r = R[s1][a]     # reward from having executed a from s1 
            temp = Q[s1][a]  # store the old Q value for comparison on convergence check
            
            
            Q[s1][a] = (1-alfa) * Q[s1][a] + alfa * (r + gamma * np.max(Q[s2])) # update the qtable               
            s1 = s2  # let the process evolve from s2 now
        
            delta_epoch = max(delta_epoch, abs(temp - Q[s1][a]))
                
        # convergence check 
        if delta_epoch < threshold and epsilon <= 0:
            if verbose: 
                print("Qtable:\n" + str(Q) + "\n")
            break            
                    
    # reconstruct the policy 
    policy = [0] * n_states  # optimal policy
    
    for s in range(n_states):
        policy[s] = np.argmax(Q[s])
            
    return policy

def flip_bised_coin(p):
    """ Return true with probability p, false with probability 1-p """
    return np.random.random() < p 

In [9]:
policy = qlearning(R, end_state, n_states, n_actions, gamma, alfa, threshold, epsilon, decay, verbose=True)
policy_readable = policy_to_string(policy, names)
print("Optimal policy is:\n" + str(policy_readable))

Qtable:
[[  0   0   0   0 396   0]
 [  0   0   0 316   0 496]
 [  0   0   0 316   0   0]
 [  0 396 252   0 396   0]
 [316   0   0 316   0 496]
 [  0 396   0   0 396 496]]

Optimal policy is:
['go 4', 'go OUT', 'go 3', 'go 1', 'go OUT', 'go OUT']
